In [ ]:
# prompt: please install all the below using pip

#!pip install numpy pandas matplotlib seaborn scikit-learn tensorflow keras clean_text pandarallel


In [60]:
import nltk
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.lm import Vocabulary
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, Normalizer
import joblib
from cleantext import clean
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from pandarallel import pandarallel
import ast
import math
from imblearn.under_sampling import RandomUnderSampler
pandarallel.initialize(progress_bar=False)
import time

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

import os
IN_COLAB = False
if os.getenv("COLAB_RELEASE_TAG"):
   IN_COLAB = True

news_processed = None
bbc_proccessed = None

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package punkt to /Users/fridakorzen-
[nltk_data]     bohr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/fridakorzen-
[nltk_data]     bohr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/fridakorzen-
[nltk_data]     bohr/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [47]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

# Part 1

### Task 1

In [63]:
#load data
dataPath = "../data/"
if IN_COLAB:
  dataPath = "/content/drive/MyDrive/"
nsdf = pd.read_csv(dataPath + "news_sample.csv")
nsdf = nsdf.reset_index(drop=True)  # Reset index??
nsdf_raw = nsdf.copy(deep=True)
nsdf.info()   # Check column types and missing values



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        250 non-null    int64  
 1   id                250 non-null    int64  
 2   domain            250 non-null    object 
 3   type              238 non-null    object 
 4   url               250 non-null    object 
 5   content           250 non-null    object 
 6   scraped_at        250 non-null    object 
 7   inserted_at       250 non-null    object 
 8   updated_at        250 non-null    object 
 9   title             250 non-null    object 
 10  authors           170 non-null    object 
 11  keywords          0 non-null      float64
 12  meta_keywords     250 non-null    object 
 13  meta_description  54 non-null     object 
 14  tags              27 non-null     object 
 15  summary           0 non-null      float64
dtypes: float64(2), int64(2), object(12)
memory u

In [49]:
# unique lable values
unique_values = nsdf['type'].unique()
print(unique_values)

['unreliable' 'fake' 'clickbait' 'conspiracy' 'reliable' 'bias' 'hate'
 'junksci' 'political' nan 'unknown']


In [50]:
#nan and unknown removed as they seem useless when training a classifier
nsdf = nsdf.dropna(subset=['type'])
nsdf = nsdf.loc[nsdf['type']!='unknown']
newunique_values = nsdf['type'].unique()
print(newunique_values)


['unreliable' 'fake' 'clickbait' 'conspiracy' 'reliable' 'bias' 'hate'
 'junksci' 'political']


In [51]:
# try:
#     nsdf.drop(columns=['Unnamed: 0', 'keywords', 'summary', 'inserted_at', 'updated_at'], inplace=True)
# except:
#     pass
# nsdf.info()

Cleaning and Preprocessing

In [52]:
def cleanText(data, column):
    data[column] = data[column].parallel_apply(clean_text_help)
    return data

def clean_text_help(text):
    if isinstance(text, str):
        # Remove excess whitespace
        text = re.sub(r"\s+", " ", text).strip()
        #replace dates
        text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])[-/.]?(0[1-9]|1[0-2])[-/.]?([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 1
        text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])\s([A-Za-z]{3})\s([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 2
        return clean(text, lower=True, no_line_breaks=True, no_numbers=True, no_emails=True, no_urls=True, no_punct=True, replace_with_url=r"__URL__", replace_with_email=r"__EMAIL__", replace_with_number=r"__NUM__", replace_with_digit=r"__NUM__")
    raise TypeError("Clean_text passed non-string")

In [53]:
#Tokenize the text function
def tokenizeText(data, column):
    def tokenize_text_help(text):
        if isinstance(text, str):
            return (word_tokenize(text))
        return text  # Return unchanged if not a string
    data[column] = data[column].parallel_apply(tokenize_text_help)  # Apply function
    return data



In [54]:
#function for removeing stopwords
def remove_stopwords_help(text):
    text = pd.Series(text)
    stop_words = set(stopwords.words('english'))  # Load stopwords
    return text[~text.isin(stop_words)].to_list()  # Remove stopwords

def remove_stopwords(data, column):
    data['content'] = data['content'].parallel_apply(remove_stopwords_help)  # Apply function
    return data

In [ ]:
#Returns a pandas series, with word and frequency, very fast.
def getFreq(data, column):
    return len(data[column].str.split().explode().value_counts())

def getFreq_tokinized(data, column):
    return len(data[column].explode().value_counts())

def Vocab_size_tokinized(data, column):
    return len(data[column].explode())

def Vocab_size(data, column):
    return len(data[column].str.split().explode())

In [56]:

#function for removeing stopwords
def dataStemming(data, column):
    ps = PorterStemmer()
    def dataStemming_help(text):
        text = pd.Series(text)
        if(isinstance(text, str)):
            return pd.Series(ps.stem(text))
        return text.apply(ps.stem).to_list()
    data[column] = data[column].parallel_apply(dataStemming_help)  # Apply function
    return data


In [ ]:
printing_copy = nsdf_raw.copy(deep=True)
freq_raw = getFreq(printing_copy, 'content')
data_size_raw = Vocab_size(printing_copy, 'content')

printing_copy = printing_copy.dropna(subset=['type'])
printing_copy = printing_copy.loc[nsdf['type']!='unknown']

freq_pre = getFreq(printing_copy, 'content')
data_size_pre = Vocab_size(printing_copy, 'content')
freq_clean = getFreq(cleanText(printing_copy, 'content'), 'content')
data_size_clean = Vocab_size(printing_copy, 'content')
freq_token = getFreq_tokinized(tokenizeText(printing_copy, 'content'), 'content')
data_size_token = Vocab_size_tokinized(printing_copy, 'content')
freq_stopwords = getFreq_tokinized(remove_stopwords(printing_copy, 'content'), 'content')
data_size_stopwords = Vocab_size_tokinized(printing_copy, 'content')
freq_stemmed = getFreq_tokinized(dataStemming(printing_copy, 'content'), 'content')
data_size_stemmed = Vocab_size_tokinized(printing_copy, 'content')


print("--- Unique words ---")
print("Raw: ", freq_raw)
print("Preprocessing: ", freq_pre)
print("After cleaning: ", freq_clean)
print("After tokenizing: ", freq_token)
print("After removing stopwords: ", freq_stopwords)
print("After stemming: ", freq_stemmed)

print(" ")

print("--- Word counts ---")
print("Raw: ", data_size_raw)
print("Preprosseing: ", data_size_pre)
print("After cleaning: ", data_size_clean)
print("After tokenizing: ", data_size_token)
print("After removing stopwords: ", data_size_stopwords)
print("After stemming: ", data_size_stemmed)


#Delete copy:
del printing_copy
del freq_pre, data_size_pre, freq_clean, data_size_clean, freq_token, data_size_token, freq_stopwords, data_size_stopwords, freq_stemmed, data_size_stemmed



--- Unique words ---
Raw:  30005
Preprocessing:  27721
After cleaning:  15649
After tokenizing:  15641
After removing stopwords:  15509
After stemming:  10486
 
--- Word counts ---
Raw:  170462
Preprosseing:  151344
After cleaning:  150431
After tokenizing:  150679
After removing stopwords:  86067
After stemming:  86067


In [27]:
# One big function to process data:
def processData(data, column):
    def apply_sequential_helper(functions):
        # assume type siganture of functions to be List[f : String -> string ]
        def inner(text):
            for f in functions:
                text = f(text)
            return text
        return inner

    def clean_text_help(text):
        if isinstance(text, str):
            # Remove excess whitespace
            text = re.sub(r"\s+", " ", text).strip()
            #replace dates
            text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])[-/.]?(0[1-9]|1[0-2])[-/.]?([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 1
            text = re.sub(r"(0[1-9]|[1-2][0-9]|3[0-1])\s([A-Za-z]{3})\s([0-9]{2}|[0-9]{4})", "<DATE>", text)  # Replace date type 2
            return clean(text, lower=True, no_line_breaks=True, no_numbers=True, no_emails=True, no_urls=True, no_punct=True, replace_with_url=r"__URL__", replace_with_email=r"__EMAIL__", replace_with_number=r"__NUM__", replace_with_digit=r"__NUM__")
        raise TypeError("Clean_text passed non-string")

    def tokenize_text_help(text):
        if isinstance(text, str):
            return pd.Series(word_tokenize(text))
        return text  # Return unchanged if not a string

    def remove_stopwords_help(text):
      # text is a Series[str]
        stop_words = set(stopwords.words('english'))  # Load stopwords
        #if isinstance(text, str):
        #    return [word for word in text.at[0, 'content'] if not word.lower() in stop_words]
        #return text  # Return unchanged if not a string
        return text[~text.isin(stop_words)]

    ps = PorterStemmer()
    def dataStemming_help(text):
        #if isinstance(text, str):
        #    return ps.stem(text)
        #return text  # Return unchanged if not a string
        if(isinstance(text, str)):
            return pd.Series(ps.stem(text))
        return text.apply(ps.stem)

    def type_cleaner(text):
        if isinstance(text, str):
            return pd.Series(text).to_list()
        return text.to_list()

    data[column] = data[column].parallel_apply(apply_sequential_helper(
        [clean_text_help, # str -> str
        tokenize_text_help, # str -> list[str]
        remove_stopwords_help, #series[str] -> series[str]
        dataStemming_help, #series[str] -> series[str]
        type_cleaner # series[str] -> series[str]
    ]))
    return data

In [64]:
"""
nsdf_cleaned = cleanText(nsdf, 'content')
nsdf_tokenized = tokenizeText(nsdf_cleaned, 'content')                  #tokenizing
nsdf_cleaned_tokenized_nostopwords = remove_stopwords(nsdf_tokenized, 'content')           #removing stopwords
nsdf_preprocessed = dataStemming(nsdf_cleaned_tokenized_nostopwords, 'content')
"""
nsdf_processed = processData(nsdf, 'content')
nsdf_processed.dropna(subset=['content'], inplace=True)  # Drop rows with no content
nsdf_processed.reset_index(drop=True, inplace=True)  # Reset index
print(nsdf_processed.at[0, 'content'])

['sometim', 'power', 'christma', 'make', 'wild', 'wonder', 'thing', 'need', 'believ', 'holi', 'triniti', 'believ', 'posit', 'power', 'good', 'other', 'simpl', 'act', 'give', 'without', 'receiv', 'lost', 'mani', 'us', 'day', 'worri', 'money', 'success', 'hold', 'us', 'back', 'give', 'other', 'need', 'one', 'congreg', 'ohio', 'move', 'action', 'power', 'sermon', 'given', 'church', 'christma', 'eve', 'pastor', 'grand', 'lake', 'unit', 'methodist', 'church', 'celina', 'ohio', 'gave', 'emot', 'sermon', 'import', 'understand', 'messag', 'jesu', 'mani', 'religi', 'peopl', 'messag', 'jesu', 'help', 'other', 'make', 'sure', 'peopl', 'suffer', 'get', 'help', 'need', 'enjoy', 'life', 'littl', 'bit', 'sermon', 'realli', 'generos', 'look', 'like', 'live', 'jesu', 'live', 'long', 'time', 'ago', 'act', 'gener', 'fashion', 'time', 'would', 'gener', 'act', 'look', 'like', 'time', 'focu', 'sermon', 'potenc', 'sermon', 'lost', 'congreg', 'move', 'take', 'action', 'sermon', 'end', 'congreg', 'decid', 'tak

In [39]:
#word frequency pre preprocessing
# print("word frequency pre preprocessing")
# word_frequency_pre = getFreq(nsdf_raw, 'content').sum()
# print(word_frequency_pre)

In [ ]:
#word frequency post preprocessing
# print("word frequency post preprocessing")
# getfreq assumes different type signature, so we have to do it this way
# word_frequency_post = nsdf_processed["content"].explode().value_counts().sum()
# print(word_frequency_post)

In [ ]:
#word frequency post stemming
#print("word frequency post stemming")
#word_frequency_postStem = getFrequency(nsdf_.processed, 'content', 1)
#print(sum(word_frequency_postStem.values()))

In [ ]:
#pre = word_frequency_pre
#post = word_frequency_post
#print("Reduction rate of the vocabulary size after removing stopwords:")
#print(abs(pre -post))
#print("Further  reduction rate of the vocabulary size after stemming")
#postStem = sum(word_frequency_post.values())
#print(abs(post -postStem))

### Task 2

In [ ]:
#load data
fakeNewsCorpus = pd.read_csv(dataPath + "995,000_rows.csv")
#Hva saten er den der unnamed???
print(fakeNewsCorpus.head())
#fakeNewsCorpus['content'].duplicated()
news_noDup = fakeNewsCorpus.drop_duplicates(subset=['content']).dropna(subset=['content']).reset_index(drop=True)

In [ ]:
#Cleaning
news_processed = processData(news_noDup, 'content')
news_processed.to_json(dataPath + "news_processed.json", orient='records', lines=True)

### Task 3

In [ ]:
if news_processed is None:
    json_reader = pd.read_json(dataPath + "news_processed.json", orient='records', lines=True, chunksize=1500)
    news_processed = pd.concat(json_reader, ignore_index=True)
# timed: 12 min på M1 macbook chunk=1000
print(news_processed.info())   # Check column types and missing values
#fndf = fakeNewsCorpus.reset_index(drop=True)  # Reset index
fndf = news_processed


### Cleaning and Preprocessing

In [ ]:
print("Pandas DataFrame:")
display(fndf.iloc[0])



## Observations about dataset

In [ ]:
unique_values = fndf['type'].unique()
print(unique_values)
#hard to know how to classify nan and unknown, so removed for now
# we also remove the a weird type 
fndf = fndf.dropna(subset=['type'])
fndf = fndf.loc[(fndf['type']!='unknown') & (fndf['type']!='unreliable') & (fndf['type'] != '2018-02-10 13:43:39.521661')]
# Need to reset index

newunique_values = fndf['type'].unique()
print(newunique_values)


In [ ]:

# groups (reliable) as truenews 1 and (all others) in fakenews 0
    #note this is naive and should be reconsidered later

# reliable, clickbait and political are all, by their contents, factually correct (albeit possibly politcally motivated)
# we deem those to be "real" news
print("adding binary labels")
fndf['type'] = fndf['type'].replace(r'^(reliable|clickbait|political)$', '1', regex=True) 
fndf['type'] = fndf['type'].replace(r'^(?!1$).+', '0', regex=True)   # Replace everything except '1' with '0'
#fndf['type'] = fndf['type'].fillna('0')

newunique_values = fndf['type'].unique()
print(newunique_values)
fndf.shape[0]
fndf['type'] = fndf['type'].astype(int)  # Convert to integer

print(("real vs fake:"))
print(fndf['type'].value_counts())

In [ ]:
relib_news = fndf.loc[fndf['type'] == 1]
fake_news = fndf.loc[fndf['type'] == 0]

print(relib_news.shape[0], " : ", fake_news.shape[0])

print("distribution for real news")
#print(relib_news["content"].explode().value_counts())
real_dist = FreqDist(relib_news["content"].explode())
print(real_dist.most_common(200))

print("distribution for fake news")
#print(fake_news["content"].explode().value_counts())
fake_dist = FreqDist(fake_news["content"].explode())
print(fake_dist.most_common(200))

print("distribution for all news")
#print(fndf["content"].explode().value_counts())
all_dist = FreqDist(fndf["content"].explode())
print(all_dist.most_common(200))
joblib.dump((real_dist, fake_dist, all_dist), dataPath + "dists.pkl")

In [ ]:

(real_dist, fake_dist, all_dist) = joblib.load(dataPath + "dists.pkl")

In [ ]:

fake_total = int(pd.Series([b for (a, b) in fake_dist.most_common(1000)]).sum())
real_total = int(pd.Series([b for (a, b) in real_dist.most_common(1000)]).sum())

print(f"real total: {real_total}, fake total: {fake_total}")
print(real_total-fake_total)

In [ ]:

real_pd = pd.Series(dict(real_dist.most_common(1000))).apply(lambda x: math.log2(x/real_total))
fake_pd = pd.Series(dict(fake_dist.most_common(1000))).apply(lambda x: math.log2(x/fake_total))
real_set = set(real_pd.index)
fake_set = set(fake_pd.index)
unique_words = real_set.union(fake_set) - real_set.intersection(fake_set)
print(f"Amount of words only present in one set than another: {len(unique_words)}")

large_diff = [(word, float(real_pd.get(word, 0) - fake_pd.get(word, 0))) for word in fake_pd.index if abs(real_pd.get(word, 0) - fake_pd.get(word, 0)) > 5]
large_diff.sort(key=lambda x: x[1], reverse=True)
print(large_diff)

print("Unique tokens:")
for token in ["url", "email", "num", "date"]:
    print(f"TOKEN: {token} - Real: {real_pd[token]}, Fake: {fake_pd[token]}")

plt.bar(real_pd.keys(), real_pd.values)
plt.show()

## Task 4

In [ ]:
# Splitting into test, train and validation
X_train_full, X_valtest_full, y_train, y_valtest = train_test_split(fndf, fndf['type'], test_size=0.2, random_state=42) # Replace with fakenews dataset for real linreg.
X_test_full, X_val_full, y_test, y_val = train_test_split(X_valtest_full, y_valtest, test_size=0.5, random_state=42)
# x_train = testing_ x, y_train = training_y
# (x_test
X_train = X_train_full['content']
X_test = X_test_full['content']
print("train size:", y_train.shape)
print("val size:", y_val.shape)
print("test size:", y_test.shape)


# Part 2

## Task 0, splitting labels into reliable and unreliable.

In [ ]:

unique_values = fndf['type'].unique()
print(unique_values)
#hard to know how to classify nan and unknown, so removed for now
# we also remove the a weird type 
fndf = fndf.dropna(subset=['type'])
fndf = fndf.loc[fndf['type']!='unknown']
# Need to reset index

newunique_values = fndf['type'].unique()
print(newunique_values)


# groups (reliable) as truenews 1 and (all others) in fakenews 0
    #note this is naive and should be reconsidered later

# reliable, clickbait and political are all, by their contents, factually correct (albeit possibly politcally motivated)
# we deem those to be "real" news
fndf['type'] = fndf['type'].replace(r'^(reliable|clickbait|political)$', '1', regex=True) 
fndf['type'] = fndf['type'].replace(r'^(?!1$).+', '0', regex=True)   # Replace everything except '1' with '0'
#fndf['type'] = fndf['type'].fillna('0')

newunique_values = fndf['type'].unique()
print(newunique_values)
fndf.shape[0]

## Task 1 - Simple linear regression model.

Get the top 10000 words, and how often they occur in each article

In [ ]:
#Standarize fndf?

# get top 10000 words for vocab in training data to avoid leaking data from test set
print("Finding vocabulary:")
vocab = X_train.explode().value_counts()[:10000].keys()
#training_split_freq = FreqDist(X_train.explode())
#vocab = [x for (x, y) in training_split_freq.most_common(10000)]
#print(pd.Series(nsdf["content"][0]).value_counts())
#print(pd.Series(fndf["content"][0]).value_counts()[top.keys()[0]])

def CountFreq(words, vocab):
    # words: Series[str]
    # vocab: list[str] of words to count
    words = pd.Series(words)
    new_row = pd.Series(np.zeros(len(vocab)))
    n = len(vocab)
    words_in_article = pd.Series(words).value_counts()
    return vocab.apply(lambda x: words_in_article.get(x, 0))

#rowsFreq = X_train.parallel_apply(lambda row: CountFreq(row, pd.Series(top.keys())))
# her bruger vi forskellige vocabularies - vi skal finde vocab for træning og apply den til test_rowsFreq
vectorizer = CountVectorizer(analyzer=lambda x: x, vocabulary=vocab)
print("vectorizing X_train")
rowsFreq = vectorizer.fit_transform(X_train)
print("vectorizing X_test")
test_rowsFreq = vectorizer.fit_transform(X_test)
print(rowsFreq)


In [ ]:
#store data

joblib.dump((rowsFreq, test_rowsFreq), dataPath + "rowsFreq.pkl")
joblib.dump((y_train, y_test, y_val), dataPath + "y.pkl")
#joblib.dump((x_undersampled, y_undersampled), dataPath + "undersampled.pkl")
joblib.dump((X_train_full, X_test_full, X_val_full), dataPath + "X_full.pkl")

Creating the linear regression

In [ ]:
#scaler = StandardScaler(with_mean=False)
scaler = Normalizer()
undersampler = RandomUnderSampler(random_state=42)

X_train_scale = scaler.fit_transform(rowsFreq)
X_test_scale = scaler.transform(test_rowsFreq)
x_undersampled, y_undersampled = undersampler.fit_resample(X_train_scale, y_train)
print("Starting model training:")
linReg = LogisticRegression(max_iter=1000, penalty="l1", solver='liblinear', random_state=42)#, class_weight="balanced")
linReg_weighed = LogisticRegression(max_iter=1000, penalty="l1", solver='liblinear', random_state=42, class_weight="balanced")
linReg_undersampled = LogisticRegression(max_iter=1000, penalty="l1", solver='liblinear', random_state=42)
print("fitting standard model")
linReg.fit(X_train_scale, y_train)
print("fitting weighted model")
linReg_weighed.fit(X_train_scale, y_train)
print("fitting undersampled model")
linReg_undersampled.fit(x_undersampled, y_undersampled)

y_pred = linReg.predict(X_test_scale)
y_pred_weighted = linReg_weighed.predict(X_test_scale)
y_pred_undersampled = linReg_undersampled.predict(X_test_scale)
f1 = f1_score(y_test, y_pred)
f1_weighted = f1_score(y_test, y_pred_weighted)
f1_undersampled = f1_score(y_test, y_pred_undersampled)
# Print results
print("Standard : Weighted : Undersampled")
print(f"F1 Score: {f1:.4f} : {f1_weighted:.4f} : {f1_undersampled:.4f}")
print(f"Hyperparameters: max_iter=1000, solver='liblinear', binary bag-of-words")


In [ ]:
# Save model
joblib.dump((linReg, linReg_weighed, linReg_undersampled), dataPath + "linReg.pkl")

In [ ]:
# Reload data 
(linReg, linReg_weighed, linReg_undersampled) = joblib.load(dataPath + "linReg.pkl")
(rowsFreq, test_rowsFreq) = joblib.load(dataPath + "rowsFreq.pkl")
(y_train, y_test, y_val) = joblib.load(dataPath + "y.pkl")
#(x_undersampled, y_undersampled) = joblib.load(dataPath + "undersampled.pkl")

scaler = Normalizer()
X_train_scale = scaler.fit_transform(rowsFreq)
X_test_scale = scaler.transform(test_rowsFreq)

y_pred = linReg.predict(X_test_scale)
y_train_pred = linReg.predict(X_train_scale)

In [ ]:
#Visualize
print(f"F1 Score: Train: {f1_score(y_train, y_train_pred):.4f}, Test: {f1_score(y_test, y_pred):.4f}")
print(f"Accuracy (test): {accuracy_score(y_test, y_pred):.4f}")

cm = confusion_matrix(y_test, y_pred, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=linReg.classes_)
disp.plot()
plt.show()

### Task 3

In [ ]:
bbc_raw = pd.from_csv(dataPath + "bbc_articles.csv")

bbc_processed = processData(bbc_raw, 'text')
bbc_processed['type'] = 1

bbc_processed.to_json(dataPath + "bbc_processed.json", orient='records', lines=True)


In [ ]:
if bbc_proccessed is None:
    json_reader = pd.read_json(dataPath + "bbc_processed.json", orient='records', lines=True, chunksize=1500)
    bbc_processed = pd.concat(json_reader, ignore_index=True)

bbcdf = bbc_processed

BBC_train = pd.concat([X_train, bbcdf['text']]).reset_index(drop=True)
BBC_train = scaler.fit_transform(BBC_train)
BBC_y = pd.concat([y_train, bbcdf["type"]])
bbcReg = LogisticRegression(max_iter=1000, penalty="l1", solver='liblinear', random_state=42)
bbcReg.fit(BBC_train, BBC_y)
BBC_pred = bbcReg.predict(X_test)
print(f"F1: {f1_score(y_test, BBC_pred)}")